In [1]:
import glob 
import time

import pandas as pd 
import json


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import warnings
warnings.filterwarnings("ignore")

from config import *

C:\Users\User\AppData\Local\Temp\ipykernel_18504\687558935.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
### Constants ###
base_url = "https://www.linkedin.com"
driver = webdriver.Chrome()
wait_time = 10



wait_and_get_element = lambda x : WebDriverWait(driver, wait_time).until(
        EC.presence_of_element_located((By.XPATH, (x)))
    )

In [3]:
### Xpaths ### 

# Login buttons
login_email = '//input[@id="session_key"]'
input_password = '//input[@id="session_password"]'
login_button = '//button[contains(text() , "identifier")]'
input_password_2 = '//input[@id="password"]'
accept_cookie_button = '//button[@action-type="ACCEPT"]'
identif_button = '(//a[contains(text() , "S’identifier")])[1]'

# Text the recruters button
message_button = '(//button[contains(@aria-label, "Envoyer un message")])[2]'
connect_button = '(//button[contains(@aria-label, "Invitez")])[2]'
confirm_invit = '(//button[contains(@aria-label, "Envoyer")])[1]'

In [4]:
# Load and prepare cookies
cookie_url = glob.glob(r"data/cookies/*linkedin*")[0]
with open(cookie_url, "r") as file:
    cookies:list = json.load(file)["cookies"]

def map_samesite_value(value):
    if value not in ["Strict", "Lax", "None"]:
        return "None"
    return value


for cookie in cookies:
    if "sameSite" in cookie:
        cookie["sameSite"] = map_samesite_value(cookie["sameSite"])

In [5]:
# Start the session
driver.get(base_url)

# Cookie feeding
for cookie in cookies:
    # Convertir expirationDate en entier, Selenium ne supporte pas les floats
    if "expirationDate" in cookie:
        cookie["expiry"] = int(cookie["expirationDate"])
        del cookie["expirationDate"]
    driver.add_cookie(cookie)


time.sleep(3)

In [6]:
# Connexion 
try:
    wait_and_get_element(accept_cookie_button).click()
except:
    pass

try:
    driver.find_element(By.XPATH , (login_button)).click()
except:
    driver.find_element(By.XPATH, (identif_button)).click()

try:
    email_input = wait_and_get_element(login_email)


    password_input = wait_and_get_element(input_password)

    email_input.send_keys(email)
    password_input.send_keys(password)

    driver.find_element(By.XPATH , (login_button)).click()
except:
    pass

time.sleep(3)


# Resend the password if needed 
try:
    driver.find_element(By.XPATH , (input_password_2)).send_keys(password)
    driver.find_element(By.XPATH , (login_button)).click()
    time.sleep(3)
except:
    pass

In [8]:
counter = 0
for link in df["LinkedIn"]:
    driver.get(link)
    
    try:
        wait_and_get_element(connect_button).click()
        wait_and_get_element(confirm_invit).click()
        counter += 1
    except:
        pass

    time.sleep(1)

print("Number of added recruters ; " , counter)

Number of added recruters ;  79
